In [51]:
# Integrate and pick out suitable PTT contents for training RNN and seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict = {'Tech_job':'1519748408.0598423','Boy-Girl':'1520323590.6503718',
              'AllTogether':'1519135097.7481742','sex':'1520269282.2598605-merge',
              'C_Chat':'1520385092.0256078-merge','movie':'1520492468.9048126-merge'}
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
remove_type = ['無題', '轉錄', '轉載', '轉發', '轉貼', '轉潑', '轉PO', '公告', '判決', '申請', '申訴', '水桶', '教學', 'ANSI', '市調', '問卷', '廣告', '協尋', '尋人']
max_word_length = 256
min_word_length = 100
max_title_length = 20
min_title_length = 4
label_newnum = len(board_name)

board_folder = []
board_dir = []
content_dir = []
push_dir = []
new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)

for name_ind in range(len(board_name)):
    # Original data path.
    board_folder.append(board_name[name_ind] + '_' + board_dict[board_name[name_ind]])
    board_dir.append(os.path.join(base_dir, board_folder[name_ind]))
    if not os.path.exists(board_dir[name_ind]):
        os.makedirs(board_dir[name_ind])
    content_dir.append(os.path.join(board_dir[name_ind], 'content'))
    if not os.path.exists(content_dir[name_ind]):
        os.makedirs(content_dir[name_ind])
    push_dir.append(os.path.join(board_dir[name_ind], 'push'))
    if not os.path.exists(push_dir[name_ind]):
        os.makedirs(push_dir[name_ind])
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

In [96]:
#整理、修改各大看版原始數據
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

board_post_num = []
board_word_num_tpye = ['min','q10','q25','q50','q75','q90','max','mean','std']
board_word_num = {'min':[],'q10':[],'q25':[],'q50':[],'q75':[],'q90':[],'max':[],'mean':[],'std':[]}
word_filters='`|＼｜＊／＝≦≧＿＠＃＄％︿＆～§◎．※↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔'

def arrange_content(content_text):
    # Remove http line in content.
    http_local = content_text.rfind('http')
    while http_local >= 0:
        http_str = content_text.rfind('\n',0 , http_local)
        if http_str < 0:
            http_str = 0
        http_end = content_text.find('\n', http_local)
        if http_end < 0:
            http_end = len(content_text)
        content_text = content_text.replace(content_text[http_str:http_end], '')
        http_local = content_text.rfind('http', 0, http_str)
    # Remove filtered CHARs in content.
    for char_filter in word_filters:
        http_local = content_text.rfind(char_filter)
        while http_local >= 0:
            content_text = content_text.replace(char_filter, '')
            http_local = content_text.rfind(char_filter, 0, http_local)
    return content_text
                        
for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(board_dir[name_ind], board_dict[board_name[name_ind]] + '.csv'))
    board_post_num.append(len(dfs))
    print("Running ..., board: ", board_name[name_ind])
    for dfs_index in range(board_post_num[name_ind]-1, -1, -1):
        read_index = int(dfs.iloc[dfs_index, 0])
        type_name = str(dfs.iloc[dfs_index, 1])
        word_length = int(dfs.iloc[dfs_index, 5])
        title_name = str(dfs.iloc[dfs_index, 9])
        title_length = len(title_name)
        if not is_continue:
            print("index: {:0>6d}".format(dfs_index), end='\r')
        # Post type filter
        is_continue = False
        for remove in remove_type:
            if type_name.find(remove)>=0:
                is_continue = True
                break
        # Remove data if the type was not required.
        if is_continue:
            dfs.drop([dfs_index], inplace=True)
            continue
        # Remove data if title length was false.
        if title_length > max_title_length or title_length < min_title_length:
            dfs.drop([dfs_index], inplace=True)
            continue
        # Remove data if content word length was false.
        if word_length > max_word_length*4 or word_length < min_word_length:
            dfs.drop([dfs_index], inplace=True)
            continue
        # Read csv and remove file which was not existed.
        text = ''
        with open(os.path.join(content_dir[name_ind], str(read_index) + '.csv'), 
                    'r', encoding = 'utf-8-sig') as file:
            csvCursor = csv.reader(file)
            for rows in csvCursor:
                for row in rows:
                    # Read content and remove empty.
                    row = arrange_content(row)
                    text = text + row
        
        # Word length filter by jieba words slit.
        text.encode('utf-8-sig')
        text_jieba = jieba.cut(text, cut_all=False)
        text = ''
        word_length = 0
        for word in text_jieba:
            word_length += 1
            text = text + ' ' + word
        # Remove data if content word length was false after jieba slitting.
        if word_length > max_word_length or word_length < min_word_length:
            dfs.drop([dfs_index], inplace=True)
            continue
        # Save posts after arrange content.
        with open(os.path.join(new_content_dir[name_ind], str(read_index) + '.csv'), 
                  'w', encoding = 'utf-8-sig') as file:
            csvCursor = csv.writer(file)
            csvCursor.writerow([text])
        dfs.iloc[dfs_index, 5] = word_length
    
    dfs.to_csv(os.path.join(totalboard_dir, board_name[name_ind] + '_fix.csv'), 
               encoding = 'utf-8-sig', index=False)
    
    board_post_num[name_ind] = (len(dfs))
    board_word_num['min'].append(dfs.iloc[:, 5].min())
    board_word_num['q10'].append(dfs.iloc[:, 5].quantile(0.1))
    board_word_num['q25'].append(dfs.iloc[:, 5].quantile(0.25))
    board_word_num['q50'].append(dfs.iloc[:, 5].quantile(0.5))
    board_word_num['q75'].append(dfs.iloc[:, 5].quantile(0.75))
    board_word_num['q90'].append(dfs.iloc[:, 5].quantile(0.9))
    board_word_num['max'].append(dfs.iloc[:, 5].max())
    board_word_num['mean'].append(dfs.iloc[:, 5].mean())
    board_word_num['std'].append(dfs.iloc[:, 5].std())
    
    print("This board was end execution, post number: ", board_post_num[name_ind])

print('Board name: ', board_name)
print('Post number: ', board_post_num)
for qtype in board_word_num_tpye:
    print(qtype, ': ', board_word_num[qtype])

Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 0.979 seconds.
Prefix dict has been built succesfully.


Running ..., board:  Tech_job
This board was end execution, post number:  14717
Running ..., board:  Boy-Girl
This board was end execution, post number:  15256
Running ..., board:  AllTogether
This board was end execution, post number:  25040
Running ..., board:  sex
This board was end execution, post number:  15491
Running ..., board:  C_Chat
This board was end execution, post number:  81274
Running ..., board:  movie
This board was end execution, post number:  30915
Board name:  ['Tech_job', 'Boy-Girl', 'AllTogether', 'sex', 'C_Chat', 'movie']
Post number:  [14717, 15256, 25040, 15491, 81274, 30915]
min :  [100, 100, 100, 100, 100, 100]
q10 :  [109.0, 116.0, 107.0, 113.0, 110.0, 111.0]
q25 :  [124.0, 139.0, 120.0, 133.0, 128.0, 127.0]
q50 :  [157.0, 177.0, 150.0, 172.0, 163.0, 162.0]
q75 :  [201.0, 216.0, 191.0, 213.0, 206.0, 205.0]
q90 :  [233.0, 240.0, 225.10000000000218, 238.0, 235.0, 235.0]
max :  [256, 256, 256, 256, 256, 256]
mean :  [163.95501800638718, 177.67665180912428, 158

In [97]:
#整合各大看板資料，挑選訓練、測試用數據
import pandas as pd
import numpy as np
import random
board_number_limit = 14400
board_post_num = []

# Select content each board by number=board_number_limit.
for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(totalboard_dir, board_name[name_ind] + '_fix.csv'))
    board_post_num.append(len(dfs))
    random_board = list(range(board_post_num[name_ind]))
    random.shuffle(random_board)
    for dfs_index in range(board_post_num[name_ind]):
        if random_board[dfs_index] >= board_number_limit:
            dfs.drop([dfs_index], inplace=True)
    dfs.to_csv(os.path.join(totalboard_dir, board_name[name_ind] + '_select.csv'), 
               encoding = 'utf-8-sig', index=False)

# Merge data each board
for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(totalboard_dir, board_name[name_ind] + '_select.csv'))
    dfs.loc[:,'board_name'] = pd.Series(np.full(board_post_num[name_ind], board_name[name_ind]))
    if name_ind > 0:
        dfs_total = pd.concat([dfs_total, dfs])
    else:
        dfs_total = dfs

# Upset and save merged data
dfs_total.reset_index(drop=True ,inplace=True)
dfs_total = dfs_total.sample(frac= 1.0 ,replace=True)
dfs_total.to_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'), 
                 encoding = 'utf-8-sig', index=False)